In [1]:
from dataLoader import MNISTDataLoader
from qiskit import Aer,transpile
import pandas as pd

from dataLoader import MNISTDataLoader,CatsDataset
from FRQI import FRQI
from NEQR import NEQR
from OQIM import OQIM

import numpy as np
from qiskit import Aer,transpile

### Parameter initialitzation

In [2]:
batch_size_train = 1
random_seed = 1
backend_sim = Aer.get_backend('qasm_simulator')
methods = ['FRQI','NEQR','OQIM']
num_pixels = [4,8,16,32,64,128,256]
dataset = 'digits'

args = {}
backend_names = []
circs = []
circ_names = []
measure_bool = True
printTime = True
outputs = []
dictionary = {}

In [3]:
input_images = []
if dataset == 'cats':
    for pixels in num_pixels:
        loader = CatsDataset('datasets/train_catvnoncat.h5')
        train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = loader.load_and_resize_images(size=(pixels, pixels))
        #image,lable = loader.get_image(train_set_x_orig[2],train_set_y_orig[2])
        input_images.append(train_set_x_orig[2])
else:
    for pixels in num_pixels:
        data_loader = MNISTDataLoader()
        train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig = data_loader.load_and_resize_images(pixels=pixels)
        #input_image,input_label= data_loader.get_image(train_set_x_orig[1],train_set_y_orig[1])
        input_images.append(train_set_x_orig[1])

### Import and visualize the image

In [4]:
# Create instances of the classes
oqim = OQIM()
neqr = NEQR()
frqi = FRQI()
circs = []
num_qubits = []
list_dic = {}
for method in methods: 
    print("SIMULATIONS WITH "+str(method)+"--------------------------------")
    list_numPixels = []
    
    for i in range(len(num_pixels)):
        circs = []
        dictionary = {}
        # Create circuits
        if method == 'FRQI':
            circuit = frqi._create_FRQI_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)
        elif method == 'NEQR':
            circuit = neqr._create_NEQR_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)
            #basis_gates = ['barrier', 'h', 'cx', 'cu1', 'id', 'x']
            #basis_gates = ['snapshot', 'delay', 'reset', 'measure', 'cx', 'h', 'barrier', 'id', 'cnot']
            #decomposed = transpile(circuit,basis_gates=basis_gates,backend=backend_sim)
            decomposed = circuit.decompose()

        elif method == 'OQIM':
            circuit = oqim.create_OQIM_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)

        print("----- Execution for an image of "+ str(num_pixels[i])+ " pixels -----")
        
        dictionary['Pixels'] = num_pixels[i]
        dictionary['Qubits'] = circuit.num_qubits
        dictionary['Depth simulation'] = circuit.depth()
        list_numPixels.append(dictionary)

    list_dic[method] = list_numPixels



SIMULATIONS WITH FRQI--------------------------------
Time needed: 1.512s for creating circuit via FRQI
----- Execution for an image of 4 pixels -----
Time needed: 1.134s for creating circuit via FRQI
----- Execution for an image of 8 pixels -----
Time needed: 5.881s for creating circuit via FRQI
----- Execution for an image of 16 pixels -----
Time needed: 26.512s for creating circuit via FRQI
----- Execution for an image of 32 pixels -----
Time needed: 115.205s for creating circuit via FRQI
----- Execution for an image of 64 pixels -----
Time needed: 535.588s for creating circuit via FRQI
----- Execution for an image of 128 pixels -----
Time needed: 3619.756s for creating circuit via FRQI
----- Execution for an image of 256 pixels -----
SIMULATIONS WITH NEQR--------------------------------
Time needed: 0.166s for creating circuit via NEQR
----- Execution for an image of 4 pixels -----
Time needed: 0.010s for creating circuit via NEQR
----- Execution for an image of 8 pixels -----
Time

In [5]:
list_dic

{'FRQI': [{'Pixels': 4, 'Qubits': 5, 'Depth simulation': 533},
  {'Pixels': 8, 'Qubits': 7, 'Depth simulation': 6225},
  {'Pixels': 16, 'Qubits': 9, 'Depth simulation': 46853},
  {'Pixels': 32, 'Qubits': 11, 'Depth simulation': 285701},
  {'Pixels': 64, 'Qubits': 13, 'Depth simulation': 1536005},
  {'Pixels': 128, 'Qubits': 15, 'Depth simulation': 7716869},
  {'Pixels': 256, 'Qubits': 17, 'Depth simulation': 37158917}],
 'NEQR': [{'Pixels': 4, 'Qubits': 12, 'Depth simulation': 76},
  {'Pixels': 8, 'Qubits': 14, 'Depth simulation': 214},
  {'Pixels': 16, 'Qubits': 16, 'Depth simulation': 856},
  {'Pixels': 32, 'Qubits': 18, 'Depth simulation': 3390},
  {'Pixels': 64, 'Qubits': 20, 'Depth simulation': 13242},
  {'Pixels': 128, 'Qubits': 22, 'Depth simulation': 53163},
  {'Pixels': 256, 'Qubits': 24, 'Depth simulation': 213063}],
 'OQIM': [{'Pixels': 4, 'Qubits': 6, 'Depth simulation': 2337},
  {'Pixels': 8, 'Qubits': 8, 'Depth simulation': 18337},
  {'Pixels': 16, 'Qubits': 10, 'Depth si

In [6]:
pd.DataFrame(list_dic['FRQI'])

,Pixels,Qubits,Depth simulation
0,4,5,533
1,8,7,6225
2,16,9,46853
3,32,11,285701
4,64,13,1536005
5,128,15,7716869
6,256,17,37158917


In [7]:
pd.DataFrame(list_dic['NEQR'])

,Pixels,Qubits,Depth simulation
0,4,12,76
1,8,14,214
2,16,16,856
3,32,18,3390
4,64,20,13242
5,128,22,53163
6,256,24,213063


In [8]:
pd.DataFrame(list_dic['OQIM'])

,Pixels,Qubits,Depth simulation
0,4,6,2337
1,8,8,18337
2,16,10,119681
3,32,12,675329
4,64,14,3487745
5,128,16,17096705
6,256,18,80969729
